In [1]:
!/usr/bin/nvidia-smi

Sun Oct 29 01:08:45 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.113.01             Driver Version: 535.113.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti     Off | 00000000:01:00.0  On |                  N/A |
|  0%   32C    P8              10W / 225W |    469MiB /  8192MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Import ALL Dependencies

In [2]:
import whisper
from whisper import load_model
import torch

In [3]:
sampleMediaDir = "/mnt/m2media/sampleMedia"

In [4]:
sampleMediaDir

'/mnt/m2media/sampleMedia'

In [5]:
!ls $sampleMediaDir

 EAH1.mp4
'How Did The Sriracha Shortage Happen_.mp4'
'How Did The Sriracha Shortage Happen_.wav'


## Set sample file to be used as audio data

In [6]:
video_path = sampleMediaDir + "/How Did The Sriracha Shortage Happen_.mp4"

In [7]:
video_path

'/mnt/m2media/sampleMedia/How Did The Sriracha Shortage Happen_.mp4'

## Check versions of pytorch and tourchaudio

In [8]:
!conda list pytorch
!conda list torchaudio


# packages in environment at /home/winadmin/miniconda3/envs/periferyNABNY2023py38:
#
# Name                    Version                   Build  Channel
pytorch                   2.2.0.dev20231028     py3.8_cpu_0    pytorch-nightly
pytorch-mutex             1.0                         cpu    pytorch-nightly
# packages in environment at /home/winadmin/miniconda3/envs/periferyNABNY2023py38:
#
# Name                    Version                   Build  Channel
torchaudio                2.2.0.dev20231028        py38_cpu    pytorch-nightly


## Convert the audio portion of a video file into a PyTorch tensor

### First we need to extract audio from the video

In [9]:
import torchaudio
from moviepy.editor import *
import os

# Generate audio file name based on the video file name
audio_file_name = os.path.splitext(os.path.basename(video_path))[0] + ".wav"

# Path to save the audio file
audio_path = os.path.join(os.path.dirname(video_path), audio_file_name)

# Check if the audio file already exists
if not os.path.exists(audio_path):
    # Load video
    video = VideoFileClip(video_path)

    # Extract audio
    audio_data = video.audio

    # Save audio
    audio_data.write_audiofile(audio_path)


### Next we can convert the audio to a Pytorch Tensor

In [10]:
audio_waveform, sample_rate = torchaudio.load(audio_path)

## Set the **device** to the CUDA device you wish to test

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


### Now we can move the audio data to the specified device

In [ ]:
audio_waveform = audio_waveform.to(device)

#### If you need help: Add this line: help(whisper)

# Check if torch is available

In [ ]:
import torch
print(torch.cuda.is_available())

# List available models

In [ ]:
available_models = whisper.available_models()
print(available_models)

# Load a model (for this example, we assume the first available model)

In [ ]:
model_name = available_models[0]
asr_model = whisper.load_model(model_name)

# Move the model to the specified device

In [ ]:
asr_model = asr_model.to(device)

# Use the moviepy library to determine the duration of the source

In [ ]:
from moviepy.editor import AudioFileClip, VideoFileClip

# Function to get duration
def get_media_duration(audio_data):
    if audio_data.endswith(('.mp4', '.mkv', '.webm', '.flv', '.mov', '.avi')):
        clip = VideoFileClip(audio_data)
    else:  # Assuming other extensions are audio files
        clip = AudioFileClip(audio_data)
    duration = clip.duration
    clip.close()
    return duration

# Get and print media duration
duration = get_media_duration(audio_data)  # Replace with the path to your audio_data
print(f"Media Duration: {duration} seconds")


# Transcribe Audio and calculate the time taken:

### (Assuming audio_data is the audio file)

In [ ]:
import time

start_time = time.time()
transcription = asr_model.transcribe(audio_data)
end_time = time.time()

processing_time = end_time - start_time


## calculate and print the performance ratio

In [ ]:
performance_ratio = duration / processing_time
media_duration_str = f"{duration:.2f}"
processing_time_str = f"{processing_time:.2f}"
performance_ratio_str = f"{performance_ratio:.2f}"

output_str = f"Media Duration: {media_duration_str} seconds | Total Processing Time = {processing_time_str} seconds | Performance: {performance_ratio_str}x real-time"
print(output_str)

## Print the Transcription

In [ ]:
print(transcription)

# Generate Report

In [ ]:
import os
import csv
import torch
from datetime import datetime

# Function to gather performance metrics (Placeholder, replace with actual metric gathering code)
def gather_metrics():
    # Replace this with code that gathers actual performance metrics
    return {
        "DateTime": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "Source File": audio_data,
        "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
        "whisper_model": model_name,
        "performance_metric": f"{performance_ratio:.2f}x real-time"
    }

# Create or open CSV file to store performance metrics
csv_file_path = "gpu_performance_metrics.csv"
file_exists = os.path.isfile(csv_file_path)
with open(csv_file_path, 'a', newline='') as csvfile:
    fieldnames = ["DateTime", "Source File", "gpu_name", "whisper_model", "performance_metric"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Write header only if the file didn't exist
    if not file_exists:
        writer.writeheader()

    # Gather performance metrics and write to CSV
    metrics = gather_metrics()
    writer.writerow(metrics)
